In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load raw dataset
df = pd.read_csv("/content/online_retail_II.csv", encoding="ISO-8859-1")

# Basic inspection
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
df.shape

(1067371, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


Standardize Column Names

This prevents downstream issues and helps SQL-style logic.

In [5]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)


In [6]:
df.columns

Index(['invoice', 'stockcode', 'description', 'quantity', 'invoicedate',
       'price', 'customer_id', 'country'],
      dtype='object')

**Convert & Validate Date Fields**


*   Enabling time-based aggregation
*   Required for impact analysis later



In [7]:
df["invoicedate"] = pd.to_datetime(df["invoicedate"])
df["invoice_year"] = df["invoicedate"].dt.year
df["invoice_month"] = df["invoicedate"].dt.to_period("M")


In [8]:
# Check missing values across columns
df.isna().sum().sort_values(ascending=False)

,0
customer_id,243007
description,4382
stockcode,0
invoice,0
invoicedate,0
quantity,0
price,0
country,0
invoice_year,0
invoice_month,0


### **Missing Value Assessment**

Missing values were reviewed to align data preparation with the project objective.

- **Customer ID:** Transactions without a customer identifier were excluded, as customer-level analysis (frequency, recency, segmentation) requires valid customer IDs.
- **Description:** A small number of missing product descriptions were observed and retained, as product text is not required for behavioral or revenue analysis.

These decisions ensure the dataset remains both analytically relevant and sufficiently large for robust customer behavior analysis.


In [9]:
# Retain only transactions with valid customer identifiers
df = df[df["customer_id"].notna()]

In [10]:
df = df.copy()
df["description"] = df["description"].fillna("Unknown Product")

In [11]:
# Handle Returns & Invalid Transactions
# Remove cancelled invoices (negative quantity or price)
df = df[(df["quantity"] > 0) & (df["price"] > 0)]


In [12]:
# Create Transaction-Level Revenue
df["revenue"] = df["quantity"] * df["price"]

In [13]:
# Focus on main market to reduce noise & size (UK dominant market)
df = df[df["country"] == "United Kingdom"]


In [14]:
df.describe(include="all").transpose()


,count,unique,top,freq,mean,min,25%,50%,75%,max,std
invoice,725250,33541,576339,542,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stockcode,725250,4616,85123A,4997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,725250,5249,WHITE HANGING HEART T-LIGHT HOLDER,4990,NaN,NaN,NaN,NaN,NaN,NaN,NaN
quantity,725250.0,NaN,NaN,NaN,12.003189,1.0,2.0,4.0,12.0,80995.0,140.796321
invoicedate,725250,NaN,NaN,NaN,2010-12-31 09:09:18.930547968,2009-12-01 07:45:00,2010-07-04 14:27:00,2010-12-02 12:30:30,2011-07-26 11:38:00,2011-12-09 12:49:00,NaN
price,725250.0,NaN,NaN,NaN,3.056777,0.001,1.25,1.95,3.75,10953.5,25.165512
customer_id,725250.0,NaN,NaN,NaN,15559.119094,12346.0,14209.0,15555.0,16938.0,18287.0,1593.844795
country,725250,1,United Kingdom,725250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
invoice_year,725250.0,NaN,NaN,NaN,2010.416095,2009.0,2010.0,2010.0,2011.0,2011.0,0.567315
invoice_month,725250,25,2011-11,58800,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.shape

(725250, 11)

In [16]:
# Save Cleaned Data
df.to_csv("processed_data.csv", index=False)


In [18]:
# Load cleaned data
df = pd.read_csv("/content/processed_data.csv", parse_dates=["invoicedate"])


In [19]:
# Define Analysis Time Grain
df["year_month"] = df["invoicedate"].dt.to_period("M")


In [20]:
# Create Customer-Month Aggregation
customer_monthly = (
    df
    .groupby(["customer_id", "year_month"])
    .agg(
        total_revenue=("revenue", "sum"),
        total_quantity=("quantity", "sum"),
        transaction_count=("invoice", "nunique")
    )
    .reset_index()
)


**Create Behavioral Metrics**

In [21]:
# calculating Average Basket Value
customer_monthly["avg_basket_value"] = (
    customer_monthly["total_revenue"] / customer_monthly["transaction_count"]
)


In [22]:
# Purchase Frequency (monthly)
customer_monthly["purchase_frequency"] = customer_monthly["transaction_count"]


In [23]:
# Identify Active vs Inactive Customers
customer_monthly["is_active"] = np.where(
    customer_monthly["total_revenue"] > 0, 1, 0
)


In [24]:
# Customer Lifetime Metrics
customer_lifetime = (
    customer_monthly
    .groupby("customer_id")
    .agg(
        lifetime_revenue=("total_revenue", "sum"),
        active_months=("year_month", "nunique"),
        avg_monthly_revenue=("total_revenue", "mean"),
        avg_basket_value=("avg_basket_value", "mean"),
        avg_purchase_frequency=("purchase_frequency", "mean")
    )
    .reset_index()
)


In [25]:
# RFM-Style Customer Segmentation
last_purchase = (
    customer_monthly
    .groupby("customer_id")["year_month"]
    .max()
    .reset_index()
)

latest_month = customer_monthly["year_month"].max()

last_purchase["recency_months"] = (
    latest_month - last_purchase["year_month"]
).apply(lambda x: x.n)


In [26]:
# Merge into lifetime table:
customer_lifetime = customer_lifetime.merge(
    last_purchase[["customer_id", "recency_months"]],
    on="customer_id",
    how="left"
)


In [27]:
# Create Value-Based Customer Segments
customer_lifetime["value_segment"] = pd.qcut(
    customer_lifetime["lifetime_revenue"],
    q=3,
    labels=["Low Value", "Medium Value", "High Value"]
)


In [28]:
# Merge Customer Segments Back to Monthly Data
customer_monthly = customer_monthly.merge(
    customer_lifetime[
        ["customer_id", "value_segment", "recency_months"]
    ],
    on="customer_id",
    how="left"
)


In [29]:
customer_monthly.head()


,customer_id,year_month,total_revenue,total_quantity,transaction_count,avg_basket_value,purchase_frequency,is_active,value_segment,recency_months
0,12346.0,2009-12,113.50,26,5,22.70,5,1,High Value,11
1,12346.0,2010-01,90.00,20,4,22.50,4,1,High Value,11
2,12346.0,2010-03,27.05,5,1,27.05,1,1,High Value,11
3,12346.0,2010-06,142.31,19,1,142.31,1,1,High Value,11
4,12346.0,2011-01,77183.60,74215,1,77183.60,1,1,High Value,11


In [30]:
customer_monthly.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
customer_id,23417.0,15524.933723,1587.147076,12346.00,14141.00,15544.00,16904.00,18287.0
total_revenue,23417.0,628.737563,1995.115897,0.85,206.21,341.86,606.46,168469.6
total_quantity,23417.0,371.751847,1267.188564,1.00,100.00,194.00,361.00,87167.0
transaction_count,23417.0,1.432335,1.211934,1.00,1.00,1.00,1.00,45.0
avg_basket_value,23417.0,411.092087,1313.300364,0.85,178.20,303.96,451.56,168469.6
purchase_frequency,23417.0,1.432335,1.211934,1.00,1.00,1.00,1.00,45.0
is_active,23417.0,1.000000,0.000000,1.00,1.00,1.00,1.00,1.0
recency_months,23417.0,3.653158,5.102975,0.00,1.00,1.00,4.00,24.0


In [31]:
# Save Outputs
customer_monthly.to_csv("customer_monthly_metrics.csv", index=False)
customer_lifetime.to_csv("customer_lifetime_metrics.csv", index=False)


In [32]:
# Define intervention start month
intervention_month = customer_monthly["year_month"].quantile(0.6)

intervention_month


Period('2011-03', 'M')

In [33]:
# Label Pre vs Post Periods
customer_monthly["period"] = np.where(
    customer_monthly["year_month"] < intervention_month,
    "Pre",
    "Post"
)


In [35]:
# Define Test vs Control Groups

customer_monthly["group"] = np.where(
    customer_monthly["value_segment"] == "High Value",
    "Test",
    "Control"
)


In [36]:
# Aggregate Metrics for Impact Comparison

impact_summary = (
    customer_monthly
    .groupby(["group", "period"])
    .agg(
        avg_revenue=("total_revenue", "mean"),
        avg_frequency=("purchase_frequency", "mean"),
        avg_basket=("avg_basket_value", "mean"),
        active_customers=("customer_id", "nunique")
    )
    .reset_index()
)

impact_summary


,group,period,avg_revenue,avg_frequency,avg_basket,active_customers
0,Control,Post,263.643630,1.117574,241.054457,2023
1,Control,Pre,264.635879,1.122642,242.862397,2532
2,Test,Post,804.081004,1.554153,499.861361,1618
3,Test,Pre,808.620070,1.605625,489.332509,1632


In [37]:
# Pre vs Post Change Calculation

impact_pivot = impact_summary.pivot(
    index="group",
    columns="period",
    values=["avg_revenue", "avg_frequency", "avg_basket"]
)

impact_pivot


avg_revenue             avg_frequency            avg_basket  \
period         Post         Pre          Post       Pre        Post   
group                                                                 
Control  263.643630  264.635879      1.117574  1.122642  241.054457   
Test     804.081004  808.620070      1.554153  1.605625  499.861361   

                     
period          Pre  
group                
Control  242.862397  
Test     489.332509

In [41]:
# # Calculate % Change

impact_change = (
    (impact_pivot.xs("Post", level=1, axis=1)
     - impact_pivot.xs("Pre", level=1, axis=1))
    / impact_pivot.xs("Pre", level=1, axis=1)
    * 100
).reset_index()

impact_change


,group,avg_revenue,avg_frequency,avg_basket
0,Control,-0.374949,-0.451369,-0.744430
1,Test,-0.561335,-3.205712,2.151676


In [42]:
# Difference-in-Differences

did_effect = (
    impact_change.loc[impact_change["group"] == "Test"]
    .drop(columns="group")
    .values
    -
    impact_change.loc[impact_change["group"] == "Control"]
    .drop(columns="group")
    .values
)

did_effect


array([[-0.18638626, -2.75434308,  2.89610648]])

In [43]:
# Sales Driver Decomposition

driver_summary = (
    customer_monthly
    .groupby(["group", "period"])
    .agg(
        customers=("customer_id", "nunique"),
        frequency=("purchase_frequency", "mean"),
        basket=("avg_basket_value", "mean")
    )
    .reset_index()
)

driver_summary


,group,period,customers,frequency,basket
0,Control,Post,2023,1.117574,241.054457
1,Control,Pre,2532,1.122642,242.862397
2,Test,Post,1618,1.554153,499.861361
3,Test,Pre,1632,1.605625,489.332509


In [44]:
# Light Statistical Validation

from scipy.stats import ttest_ind

test_pre = customer_monthly[
    (customer_monthly["group"] == "Test") &
    (customer_monthly["period"] == "Pre")
]["total_revenue"]

test_post = customer_monthly[
    (customer_monthly["group"] == "Test") &
    (customer_monthly["period"] == "Post")
]["total_revenue"]

ttest_ind(test_post, test_pre, equal_var=False)


TtestResult(statistic=np.float64(-0.10849597965875182), pvalue=np.float64(0.9136043535791808), df=np.float64(10444.601302116467))